In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import pandas as pd
import time

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.mars_db
collection = db.data_scrape_col

url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest" 

#response = requests.get(url)
#soup = bs(response.text, 'html.parser')
    
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

time.sleep(5)

soup = bs(browser.html, 'html.parser')

In [ ]:
#Careful running this.  It can slow down your computer and takes a while to load.  It's a lot.
#print(soup.prettify())

In [3]:
results = soup.find('li', class_ = "slide")
results.prettify()

'<li class="slide">\n <div class="image_and_description_container">\n  <a href="/news/8371/curiosity-on-the-move-again/" target="_self">\n   <div class="rollover_description">\n    <div class="rollover_description_inner">\n     NASA\'s Mars Curiosity rover drove about 197 feet over the weekend to a site called Lake Orcadie, pushing its total odometry to over 12 miles.\n    </div>\n    <div class="overlay_arrow">\n     <img alt="More" src="/assets/overlay-arrow.png"/>\n    </div>\n   </div>\n   <div class="list_image">\n    <img alt="A self-portrait of NASA\'s Curiosity rover taken on Sol 2082 (June 15, 2018). A Martian dust storm has reduced sunlight and visibility at the rover\'s location in Gale Crater." src="/system/news_items/list_view_images/8371_PIA22486_320x240.jpg"/>\n   </div>\n   <div class="bottom_gradient">\n    <div>\n     <h3>\n      Curiosity on the Move Again\n     </h3>\n    </div>\n   </div>\n  </a>\n  <div class="list_text">\n   <div class="list_date">\n    November 

In [4]:
news_title = results.find('div', class_ = 'content_title').text
news_p = results.find('div', class_ = 'article_teaser_body').text

In [5]:
images_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(images_url)

time.sleep(5)

browser.click_link_by_partial_text('FULL IMAGE')

time.sleep(5)

soup = bs(browser.html, 'html.parser')

In [6]:
featured_image_url = "https://www.jpl.nasa.gov" + soup.find('img', class_ = "fancybox-image")['src']
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17172_ip.jpg'

In [7]:
weather_tweet_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_tweet_url)

time.sleep(5)

soup = bs(browser.html, 'html.parser')

In [8]:
mars_weather = soup.find('p', class_ = "tweet-text").text
mars_weather

'Sol 2224 (2018-11-08), high 0C/32F, low -72C/-97F, pressure at 8.65 hPa, daylight 06:19-18:36'

In [9]:
mars_facts_url = "https://space-facts.com/mars/"
browser.visit(mars_facts_url)

time.sleep(5)

tables = pd.read_html(mars_facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [10]:
df = tables[0]
df.columns = ["Statistic Name", "Stats"]
df.set_index("Statistic Name", inplace = True)
df

,Stats
Statistic Name,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [11]:
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Stats</th>\n    </tr>\n    <tr>\n      <th>Statistic Name</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </

In [12]:
html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Stats</th>    </tr>    <tr>      <th>Statistic Name</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [13]:
hemispheres_base_url = "https://astrogeology.usgs.gov/search/map/Mars/Viking/"
hemisphere_image_base_url = "https://astrogeology.usgs.gov"
hemispheres = ["cerberus_enhanced", "schiaparelli_enhanced", "syrtis_major_enhanced", "valles_marineris_enhanced"]
hemisphere_image_url = []

#Test Code
#browser.visit(hemispheres_base_url + hemispheres[0])
#time.sleep(5)
#soup = bs(browser.html, 'html.parser')
#hemisphere_name = soup.find('h2', class_ = "title").text
#hemisphere_img = soup.find('img', class_ = "wide-image")['src']
#hemisphere_img

for hemisphere in hemispheres:
    browser.visit(hemispheres_base_url + hemisphere)
    
    time.sleep(5)
    
    soup = bs(browser.html, 'html.parser')
    
    hemisphere_name = soup.find('h2', class_ = "title").text
    hemisphere_img = soup.find('img', class_ = "wide-image")['src']
    
    hemisphere_dict = {"title": hemisphere_name, "image_url": hemisphere_image_base_url + hemisphere_img}
    
    hemisphere_image_url.append(hemisphere_dict)

In [14]:
hemisphere_image_url

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

In [15]:
def scrape():
    title = news_title
    text = news_p
    main_image_url = featured_image_url
    weather = mars_weather
    stat_table = html_table

    listings = {"title":title,
                    "text":text,
                    "main_image":main_image_url,
                    "weather":weather,
                    "stat_table":stat_table,
                    "hemisphere_images":hemisphere_image_url}

    return listings

In [17]:
collection.insert_one(scrape())